In [10]:
import os
import json
import re
import nltk
from uuid import uuid4 

In [11]:
# set dir name

#inport settings
sample = False

#set input and output dir
if sample:
    input_dir = 'sample/1_json'
    output_dir = 'sample/2_clean_json'
else:
    input_dir = '../data/result/1_json'
    output_dir = '../model_training/data'
    
#set ouput path if not exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [12]:
total_files_loaded = 0
processed_files_count = 0
total_sent_count = 0


In [13]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [14]:
#helper 
def clean_sent(sent):
    sent = re.sub(r'\[(G?REF):.*\]', r'[\g<1>]', sent)
    sent = re.sub(r'[^a-zA-Z0-9\s.,;!-_+*/%&?{([\])}]','', sent)
    return sent

In [16]:
#total files in input dir
total_files_loaded = len(os.listdir(input_dir))

full_data_dict = {}

#iterate over all files
for i, filename in enumerate(os.listdir(input_dir)):
    #open document if .json
    if filename.endswith('.json'):  
        input_file_path = os.path.join(input_dir, filename)
        with open(input_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # skipp documents with faulty meta data or insufficient size
        if data['title'] is None or \
            len(data['authors']) < 1 or \
            len(data['sections']) < 3:
                continue
        processed_files_count += 1
        doc_id = 'ACL_'+'_'.join(filename.split('.')[:-2])
        sent_dict = {}
        for section in data['sections']:
            if len(section) < 2: continue
            sent_dict['-'.join([doc_id,str(len(sent_dict))])] = '**' + section['section_name'] + '**' if section['section_name'] else '****'
            for par in section['paragraphs']:
                sents = nltk.sent_tokenize(clean_sent(par))
                if len(sents) < 3: continue
                sent_dict['-'.join([doc_id,str(len(sent_dict))])] = '&&&&'
                for sent in sents:
                    sent_dict['-'.join([doc_id,str(len(sent_dict))])] = sent
                    total_sent_count += 1
        full_data_dict[doc_id] = sent_dict

In [17]:
# save file
output_file_path = os.path.join(output_dir, '/raid/deallab/CCE_Data/2p_full_data.json')     
with open(output_file_path, 'w', encoding='utf-8') as outfile:
    json.dump(full_data_dict, outfile, indent=4, ensure_ascii=False)


In [18]:
print(f"Total loaded files: {total_files_loaded}")
print(f"Total processed files: {processed_files_count}")
print(f"Total Sentence count: {total_sent_count}")

Total loaded files: 71641
Total processed files: 69953
Total Sentence count: 7555500
